In [4]:
%pip install opencv-python 
import cv2, os

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Carrega arquivo e converte para tons de cinza
i = cv2.imread('lebron.jpg')
iPB = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
#Criação do detector de faces
df = cv2.CascadeClassifier('classifier.xml')
#Executa a detecção
faces = df.detectMultiScale(iPB, scaleFactor = 1.05, minNeighbors = 7, 
    minSize = (30,30), flags = cv2.CASCADE_SCALE_IMAGE)
#Desenha retangulos amarelos na iamgem original (colorida)
for (x, y, w, h) in faces:
    cv2.rectangle(i, (x, y), (x + w, y + h), (0, 255, 255), 7)
#Exibe imagem. Título da janela exibe número de faces
cv2.imshow(str(len(faces))+' face(s) encontrada(s).', i)
cv2.waitKey(0)

-1

In [6]:
# Carrega arquivo e converte para tons de cinza
i = cv2.imread('lebron.jpg')
iPB = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)

# Criação do detector de faces
df = cv2.CascadeClassifier('classifier.xml')

# Executa a detecção
faces = df.detectMultiScale(iPB, scaleFactor = 1.05, minNeighbors = 7, minSize = (30,30), flags = cv2.CASCADE_SCALE_IMAGE)

# Cria pasta para salvar as faces
output_dir = 'faces_detectadas'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Salva cada face detectada na nova pasta e desenha retangulos amarelos na imagem original
for i, (x, y, w, h) in enumerate(faces):
    face = iPB[y:y+h, x:x+w]
    face_filename = os.path.join(output_dir, f'face_{i}.jpg')
    cv2.imwrite(face_filename, face)
    cv2.rectangle(i, (x, y), (x + w, y + h), (0, 255, 255), 7)

# Exibe imagem. Título da janela exibe número de faces
cv2.imshow(str(len(faces))+' face(s) encontrada(s).', i)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [10]:
# Caminho da pasta com as imagens
input_dir = 'faces'
output_dir = 'faces_detectadas'

# Cria a pasta para salvar as faces se não existir
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Inicializa o detector de faces
df = cv2.CascadeClassifier('classifier.xml')

# Itera sobre todas as imagens na pasta de entrada
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        file_path = os.path.join(input_dir, filename)
        i = cv2.imread(file_path)
        iPB = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)

        # Executa a detecção de faces
        faces = df.detectMultiScale(iPB, scaleFactor=1.05, minNeighbors=7, minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)

        # Salva cada face detectada na nova pasta
        for j, (x, y, w, h) in enumerate(faces):
            face = iPB[y:y+h, x:x+w]
            face_filename = os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_face_{j}.jpg')
            cv2.imwrite(face_filename, face)
            cv2.rectangle(i, (x, y), (x + w, y + h), (0, 255, 255), 7)

        # Exibe a imagem com as faces detectadas
        cv2.imshow(f'{filename} - {len(faces)} face(s) encontrada(s)', i)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
